In [ ]:
!pip uninstall -y statsmodels
!pip install statsmodels
!pip install pmdarima
!pip install geopandas
# !pip uninstall -y sklearn
# !pip install sklearn

Found existing installation: statsmodels 0.10.2
Uninstalling statsmodels-0.10.2:
  Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 9.8 MB 14.4 MB/s 
     |████████████████████████████████| 1.4 MB 12.2 MB/s 
     |████████████████████████████████| 1.0 MB 15.3 MB/s 
     |████████████████████████████████| 15.4 MB 40 kB/s 
     |████████████████████████████████| 6.3 MB 19.9 MB/s 


In [ ]:
# !pip install dtaidistance
!pip install nco
!pip install climate-indices
!pip install geopandas
!pip install metpy
!pip install fiona
!pip install geopy

     |████████████████████████████████| 70 kB 5.2 MB/s 
     |████████████████████████████████| 361 kB 15.4 MB/s 
     |████████████████████████████████| 209 kB 82.7 MB/s 


In [ ]:
!pip install pymannkendall
import pymannkendall as mk

In [ ]:
import geopandas as gpd
import xarray as xr
import pandas as pd
import numpy as np
import metpy

temp_pr = (xr.open_dataset('/content/drive/MyDrive/Colab Notebooks/cru_temp.nc').to_dataframe().dropna().drop(columns='stn').reset_index().merge(
           xr.open_dataset('/content/drive/MyDrive/Colab Notebooks/cru_precip.nc').to_dataframe().dropna().drop(columns='stn').reset_index(), on = ['lat','lon', 'time'])
)

grid = temp_pr[['lat', 'lon']].drop_duplicates().reset_index(drop=True)
grid['division'] = grid.index

temp_pr = temp_pr.merge(grid, on = ['lat', 'lon'])

In [ ]:
temp_pr_xarray = temp_pr.loc[:,['division', 'time', 'tmp', 'pre']].set_index(['division', 'time']).to_xarray()
temp_pr_xarray['lat'] = (('division'), grid['lat'])
temp_pr_xarray['lon'] = (('division'), grid['lon'])

temp_pr_xarray['pre'] = temp_pr_xarray['pre'] * metpy.units.units.millimeter
temp_pr_xarray['tmp'] = temp_pr_xarray['tmp'] * metpy.units.units.degC

temp_pr_xarray['pre'].attrs = {'units':'millimeter'}
temp_pr_xarray['tmp'].attrs = {'units':'Celsius'}
temp_pr_xarray['lat'].attrs = {'units':'degrees north'}
temp_pr_xarray['lon'].attrs = {'units':'degrees east'}

temp_pr_xarray

Magnitude,[[26.899999618530273 28.0 27.399999618530273 ... 26.700000762939453 28.100000381469727 28.200000762939453] [26.600000381469727 27.700000762939453 27.100000381469727 ... 26.600000381469727 27.80000114440918 27.80000114440918] [26.5 27.600000381469727 27.0 ... 26.5 27.700000762939453 27.600000381469727] ... [21.399999618530273 24.30000114440918 28.5 ... 30.0 25.5 22.200000762939453] [22.700000762939453 25.30000114440918 29.600000381469727 ... 30.899999618530273 26.700000762939453 23.80000114440918] [21.100000381469727 24.399999618530273 28.5 ... 29.700000762939453 25.399999618530273 22.30000114440918]]
Units,degree_Celsius
Magnitude,[[50.900001525878906 42.5 164.10000610351562 ... 243.40000915527344 103.4000015258789 54.10000228881836] [50.60000228881836 44.900001525878906 173.5 ... 275.1000061035156 103.30000305175781 46.20000076293945] [56.79999923706055 34.5 182.40000915527344 ... 278.1000061035156 101.4000015258789 51.400001525878906] ... [0.0 0.0 0.0 ... 6.5 0.0 0.0] [0.0 0.0 0.0 ... 6.300000190734863 0.0 0.0] [0.0 0.0 0.0 ... 4.0 0.0 0.0]]
Units,millimeter


In [ ]:
temp_pr_xarray.to_netcdf('/content/trial.nc')

In [ ]:
!process_climate_indices --index scaled --periodicity monthly --netcdf_temp /content/trial.nc --var_name_temp tmp --netcdf_precip /content/trial.nc --var_name_precip pre --calibration_start_year 1901 --calibration_end_year 2020 --output_file_base /content/results --scales 3

2021-10-27  12:52:30 INFO Start time:    2021-10-27 12:52:30.221960
2021-10-27  12:52:30 INFO Computing 3-month SPI/Pearson
2021-10-27  12:52:39 INFO Computing 3-month SPI/Gamma
2021-10-27  12:52:44 INFO Computing PET
2021-10-27  12:52:52 INFO Computing 3-month SPEI/Pearson
2021-10-27  12:53:01 INFO Computing 3-month SPEI/Gamma
2021-10-27  12:53:07 INFO Computing 3-month PNP
2021-10-27  12:53:09 INFO End time:      2021-10-27 12:53:09.371690
2021-10-27  12:53:09 INFO Elapsed time:  0:00:39.149730


In [ ]:
spei_gamma = xr.open_dataset('/content/results_spei_gamma_03.nc').to_dataframe().reset_index()
spei_gamma

,division,time,spei_gamma_03
0,0,1901-01-16,NaN
1,0,1901-02-15,NaN
2,0,1901-03-16,0.639134
3,0,1901-04-16,0.298051
4,0,1901-05-16,0.719678
...,...,...,...
636475,441,2020-08-16,0.189102
636476,441,2020-09-16,0.989120
636477,441,2020-10-16,0.912597
636478,441,2020-11-16,0.192242


In [ ]:
def make_test_stats(df, var, grid_var):
  df = df[[var, 'division', 'time']].copy()

  df = (
      df.groupby(['division', df['time'].dt.month_name()]).apply(lambda x:
      x.set_index('time').rolling(50).mean().shift(-49*12, 'M').dropna().reset_index()[0::10] )
  ).drop(columns='division').droplevel(2)

  df.index.set_names(['division', 'month'], inplace=True)
  df.reset_index(inplace=True)

  df = (
      df.groupby(['division', 'month']).apply(lambda x: pd.Series(mk.trend_free_pre_whitening_modification_test(x.set_index('time')[var]), 
                                                                index=['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']) ).reset_index()
  )
  df = df.merge(grid_var, on ='division')

  return df


In [ ]:
historical_tmp = make_test_stats(temp_pr, 'tmp', grid)
historical_tmp

,division,month,trend,h,p,z,Tau,s,var_s,slope,intercept,lat,lon
0,0,April,no trend,False,0.229556,1.201503,0.428571,9.0,44.333333,0.007000,27.757500,4.75,5.25
1,0,August,no trend,False,0.367521,0.901127,0.333333,7.0,44.333333,0.001500,25.115751,4.75,5.25
2,0,December,no trend,False,0.367521,0.901127,0.333333,7.0,44.333333,0.016333,27.110834,4.75,5.25
3,0,February,no trend,False,0.548006,0.600751,0.238095,5.0,44.333333,0.003000,28.232501,4.75,5.25
4,0,January,increasing,True,0.016261,2.403006,0.809524,17.0,44.333333,0.018500,27.122250,4.75,5.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,441,March,no trend,False,1.000000,0.000000,-0.047619,-1.0,44.333333,-0.030500,28.928750,13.75,14.25
5300,441,May,no trend,False,0.071505,1.802254,0.619048,13.0,44.333333,0.030533,31.865134,13.75,14.25
5301,441,November,no trend,False,0.367521,-0.901127,-0.333333,-7.0,44.333333,-0.056667,26.154334,13.75,14.25
5302,441,October,no trend,False,0.133128,1.501879,0.523810,11.0,44.333333,0.006000,29.011000,13.75,14.25


In [ ]:
historical_tmp.to_csv('historical_tmp.csv', index=False)

In [ ]:
historical_pr = make_test_stats(temp_pr, 'pre', grid)
historical_pr.to_csv('historical_pr.csv', index=False)

In [ ]:
historical_spei_gamma = make_test_stats(spei_gamma, 'spei_gamma_03', grid)
historical_spei_gamma

,division,month,trend,h,p,z,Tau,s,var_s,slope,intercept,lat,lon
0,0,April,decreasing,True,0.006864,-2.703381,-0.904762,-19.0,44.333333,-0.085543,0.424199,4.75,5.25
1,0,August,no trend,False,0.133128,1.501879,0.523810,11.0,44.333333,0.052180,-0.181841,4.75,5.25
2,0,December,decreasing,True,0.006864,-2.703381,-0.904762,-19.0,44.333333,-0.122588,0.536429,4.75,5.25
3,0,February,decreasing,True,0.008535,-2.630142,-1.000000,-15.0,28.333333,-0.067536,0.391281,4.75,5.25
4,0,January,decreasing,True,0.008535,-2.630142,-1.000000,-15.0,28.333333,-0.135290,0.727714,4.75,5.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,441,March,no trend,False,0.763891,0.300376,0.142857,3.0,44.333333,0.008695,0.010509,13.75,14.25
5300,441,May,no trend,False,0.229556,-1.201503,-0.428571,-9.0,44.333333,-0.038089,0.362815,13.75,14.25
5301,441,November,no trend,False,0.548006,-0.600751,-0.238095,-5.0,44.333333,0.019690,0.007478,13.75,14.25
5302,441,October,decreasing,True,0.006864,-2.703381,-0.904762,-19.0,44.333333,-0.110256,0.582792,13.75,14.25


In [ ]:
historical_spei_gamma.to_csv('historical_spei_gamma.csv', index=False)